# by *convention* notebooks should be for humans.

__rites.markdown__ allows an author to write code in Markdown that evaluates __code_block__s a normal.

In [1]:
    from collections import UserList
    from IPython import get_ipython
    from IPython.core.interactiveshell import InteractiveShell
    import CommonMark
    from IPython.core.inputsplitter import IPythonInputSplitter
    from IPython.core.inputtransformer import InputTransformer
    from textwrap import indent, dedent

## Markdown to code

Markdown is converted to code by persisting all code_blocks and replacing all non-code lines with blank lines.  The blank line replacement assures the right line numbering.

In [2]:
    def codify(source, parser=CommonMark.Parser, str="""""") -> str:
        """Convert markdown to code with the correct line numbers and positions in a string.

        This function replaces non-code_block nodes with blanklines."""
        for node, _ in parser().parse(source).walker():
            if node.t == 'code_block':
                line, col = node.sourcepos[0]
                end = line + len(node.literal.splitlines())
                lines = node.literal.splitlines()
                while len(str.splitlines()) < (line-1): str += '\n'
                str += indent(node.literal, ' '*col)
        return str

## Python line transformer

In [3]:
    class Markdown(UserList, InputTransformer):
        def push(self, object): 
            return self.append(object)
       
        def load(self):
            ip = get_ipython() or InteractiveShell()
            if ip.input_transformer_manager.physical_line_transforms:
                self.input_transformer_manager = ip.input_transformer_manager
                ip.input_transformer_manager = IPythonInputSplitter(
                    logical_line_transforms=[], physical_line_transforms=[], python_line_transforms=[])
            else:
                self.input_transformer_manager = IPythonInputSplitter()
            ip.input_transformer_manager.python_line_transforms = [
                object for object in ip.input_transformer_manager.python_line_transforms
                if not isinstance(object, Markdown)
            ] + [self]
            return self

        def reset(self):
            str, self.data = '\n'.join(self), []
            if str and str.splitlines()[0].strip():
                from IPython.display import display, Markdown
                display(Markdown(str))
            return dedent(self.input_transformer_manager.transform_cell(codify(str)))

## Preprocessor

In [4]:
    from nbconvert.preprocessors import Preprocessor
    class Normalize(Preprocessor):
        def preprocess_cell(Normalize, cell, resources={}, index=0):
            if cell['cell_type'] == 'code':
                cell.source = Markdown(cell['source']).reset()
            return cell, resources

## IPython extensions

    %unload_ext rites.markdown

In [5]:
    def unload_ipython_extension(ip=get_ipython()):
        for object in ip.input_transformer_manager.python_line_transforms:
            if isinstance(object, Markdown):
                ip.input_transformer_manager = object.input_transformer_manager
                break
    

    %reload_ext rites.markdown

In [6]:
    def load_ipython_extension(ip=None): Markdown().load()

## Testing

In [7]:
    class Test(__import__('unittest').TestCase): 
        def setUp(Test):
            %reload_ext rites
            load_ipython_extension()
            from nbformat import write, v4
            with open('test_markdown.ipynb', 'w') as file:
                write(v4.new_notebook(cells=[v4.new_code_cell("""Some paragraph\n\n    a=42""")]), file)
                
        def runTest(Test):
            global test_markdown
            import test_markdown
            assert test_markdown.__file__.endswith('.ipynb')
            assert test_markdown.a is 42
            
        def tearDown(Test):
            %unload_ext rites
            %rm test_markdown.ipynb
            unload_ipython_extension()

In [8]:
    if __name__ == '__main__': __import__('unittest').TextTestRunner().run(Test())

.
----------------------------------------------------------------------
Ran 1 test in 0.288s

OK
